In [1]:
import numpy as np
import pandas as pd
from simulate import TaxiSimulator

In [ ]:
# Simulation parameters
Delta = 20 # in minutes
T = int(24 * (60 // Delta))

Use synthetic data

In [ ]:
# np.random.seed(42)
# R = 3     # 3 regions
# N = 20    # 20 vehicles

# # Rider arrival rates: 10 to 30 riders per hour during busy times
# lambda_ = np.zeros((T, R))
# for t in range(T):
#     if 16 <= t % T <= 36:  # "Daytime"
#         lambda_[t] = np.random.uniform(15, 30, R)   # Busy hours
#     else:
#         lambda_[t] = np.random.uniform(2, 8, R)     # Nighttime, low demand

# # Travel time rates: Expectation between 5 to 15 minutes (1/12 to 1/4 hours)
# mu_ = np.random.uniform(4, 12, size=(T, R, R))  # 1/μ gives ~5-15 mins expected travel

# # Random valid P and Q matrices
# P = np.array([[np.random.dirichlet(np.ones(R)) for _ in range(R)] for _ in range(T)])
# Q = np.array([[np.random.dirichlet(np.ones(R)) for _ in range(R)] for _ in range(T)])

Use actual parameters

In [ ]:
with np.load('trip_counts.npz') as data:
    trip_counts = data['trip_counts']
    num_dates = data['num_dates']

with np.load('mu_cp.npz') as data:
    mu = data['mu']
    
# mask trip_counts by 1 where 0
trip_counts[trip_counts == 0] = 1

# compute arrival rate
lambda_ = trip_counts.sum(axis=2) / (Delta / 60 * num_dates)

# normalize trip_counts
P = trip_counts / trip_counts.sum(axis=2, keepdims=True)

# load time-dependent relocation matrix
with np.load('Qs.npz') as data:
    Q = data['Qs']

In [ ]:
max_time = 72  # Simulate 72 hours

# Initialize and run the simulator
sim = TaxiSimulator(T=T, R=R, N=N, lambda_=lambda_, mu_=mu_, P=P, Q=Q)
sim.run(max_time=max_time)

# Convert log to DataFrame
df_log = pd.DataFrame(sim.logger)

# Basic validation outputs
event_counts = df_log['event_type'].value_counts()

# Vehicle distribution at the end
vehicle_locations = [v.location for v in sim.vehicles]
vehicle_distribution = pd.Series(vehicle_locations).value_counts().sort_index()

In [5]:
df_log

,time,event_type,data
0,0.132258,rider_arrival,"{'region': 2, 'origin_time_block': 0}"
1,0.132258,ride_start,"{'vehicle_id': 3, 'origin': 2, 'destination': ..."
2,0.226485,rider_arrival,"{'region': 0, 'origin_time_block': 0}"
3,0.226485,ride_start,"{'vehicle_id': 1, 'origin': 0, 'destination': ..."
4,0.263728,rider_arrival,"{'region': 1, 'origin_time_block': 0}"
...,...,...,...
5418,35.937708,rider_arrival,"{'region': 0, 'origin_time_block': 23}"
5419,35.953520,relocation_completion,"{'vehicle_id': 18, 'origin': 0, 'destination': 2}"
5420,35.959506,ride_completion,"{'vehicle_id': 0, 'origin': 0, 'destination': 2}"
5421,36.009215,rider_lost,{'region': 0}
